In [1]:
import tensorflow.compat.v1 as tf

In [2]:
class Dense(object):
    def __init__(self, dim_in, dim_out, name='dense'):
        self.dim_in = dim_in
        self.dim_out = dim_out
        with tf.variable_scope(name): #规定变量名的作用域
            self.kernel = tf.get_variable('kernel', [dim_in, dim_out]) #创建变量kernel
            self.bias = tf.Variable(tf.zeros([dim_out]), name='bias') #初始化bias为dim_out维的0向量
    
    def __call__(self, inputs, activation=lambda a: a): # 接受参数，执行表达式，返回结果 -- 返回a的值
        return activation(tf.matmul(inputs, self.kernel) + self.bias) #将输入与kernel相乘加上bias

    def get_weights(self):
        return {self.kernel.name: self.kernel} #返回kernel

    def get_biases(self):
        return {self.bias.name: self.bias} #返回bias



In [3]:
class GCN(Dense):
    # https://tkipf.github.io/graph-convolutional-networks/
    # https://arxiv.org/pdf/1609.02907.pdf
    def __init__(self, dim_in, dim_out, name='gcn'):
        super(GCN, self).__init__(dim_in, dim_out, name=name)
    
    def __call__(self, inputs, activation=lambda a: a, mask=1.0, sparse=False):
        if sparse and mask != 1.0:
            self.degree = tf.sparse.reduce_sum(mask, axis=1, keepdims=True)**-0.5 
            #reduce_sum:压缩求和，用于降维
            #按行求和，且保留维度
            self.alpha = mask*self.degree*tf.transpose(self.degree) 
            #transpose:转置
            #将mask的变成self.degree*self.degree维矩阵
            return activation(tf.sparse.matmul(self.alpha, tf.matmul(inputs, self.kernel)) + self.bias)
        self.degree = tf.reduce_sum(mask, axis=1, keepdims=True)**-0.5
        self.alpha = mask*self.degree*tf.transpose(self.degree)
        return activation(tf.matmul(self.alpha, tf.matmul(inputs, self.kernel)) + self.bias)

In [4]:
class GATHead(Dense):
    # https://github.com/PetarV-/GAT/blob/master/utils/layers.py
    def __init__(self, dim_in, dim_out, residual=False, name='gathead'):
        #super:调用父类
        super(GATHead, self).__init__(dim_in, dim_out, name=name)
        with tf.variable_scope(name):
            self.attention_self = Dense(dim_out, 1, name='self')
            self.attention_other = Dense(dim_out, 1, name='other')
            self.residual = Dense(dim_in, dim_out, name='residual') if residual else None #残差

    def __call__(self, inputs, activation=lambda a: a, mask=1.0, sparse=False):
        x = tf.matmul(inputs, self.kernel)
        u, v = self.attention_self(x), self.attention_other(x)
        if sparse and mask != 1.0:
            logits = tf.sparse.add(mask*u, mask*tf.transpose(v))
            logits = tf.SparseTensor(logits.indices, tf.nn.leaky_relu(logits.values), logits.shape)
            #线性整流函数
            #tf.nn.relu()函数的目的是，将输入小于0的值幅值为0，输入大于0的值不变。
            self.alpha = tf.sparse.softmax(logits)
            #将 softmax 应用于批量的 N 维 SparseTensor.
            #softmax：归一化指数函数
            out = tf.sparse.matmul(self.alpha, x) + self.bias
        else:
            logits = tf.nn.leaky_relu(u + tf.transpose(v))
            self.alpha = mask*tf.exp(logits - tf.reduce_max(logits, 1)) #降维，按行取最大值
            self.alpha /= tf.reduce_sum(self.alpha, 1, keepdims=True) #与上一行一起相当于归一化指数函数
            out = tf.matmul(self.alpha, x) + self.bias
        return activation(out if self.residual is None else out + self.residual(inputs))

    def get_weights(self): 
        weights = {
            self.kernel.name: self.kernel,
            self.attention_self.kernel.name: self.attention_self.kernel,
            self.attention_other.kernel.name: self.attention_other.kernel,
        }
        if self.residual is not None:
            weights[self.residual.kernel.name] = self.residual.kernel
        return weights

    def get_biases(self):
        biases = {
            self.bias.name: self.bias,
            self.attention_self.bias.name: self.attention_self.bias,
            self.attention_other.bias.name: self.attention_other.bias,
        }
        if self.residual is not None:
            biases[self.residual.bias.name] = self.residual.bias
        return biases

In [5]:
class GAT(object):
    # https://arxiv.org/pdf/1710.10903.pdf
    def __init__(self, dim_in, dim_out, n_head, residual=False, concat=True, name='gat'):
        with tf.variable_scope(name):
            self.heads = [GATHead(dim_in, dim_out, residual=residual, name=str(i)) for i in range(n_head)]
        self.dim_in = dim_in
        self.n_head = n_head
        self.concat = concat
        self.dim_out = dim_out*n_head if concat else dim_out

    def __call__(self, inputs, activation=lambda a: a, mask=1.0, sparse=False):
        if self.concat:
            return tf.concat([h(inputs, activation=activation, mask=mask, sparse=sparse) for h in self.heads], 1)
        return activation(tf.reduce_mean([h(inputs, mask=mask, sparse=sparse) for h in self.heads], 0))

    def get_weights(self):
        weights = {}
        for h in self.heads:
            weights.update(h.get_weights())
        return weights

    def get_biases(self):
        biases = {}
        for h in self.heads:
            biases.update(h.get_biases())
        return biases

